In [1]:
# import libraries
import pandas as pd
import numpy as np

## job class
each job requires input:
* job name
* the model type of the job
* what machine(s) is this job using (or possibly use)

each job contains fields:
* name: job name
* model: the model type of the job
* steps: what machine(s) is this job using (or possibly use)
* status: which step is this job currently on. -1 is not assigned yet, i=0,..,len(steps)-1 is working on step[0] or waiting for it to start
* current_machine: the machine this job is assigned to or working on, if any

In [2]:
class job:
    def __init__(self, job_name, model, use_machines):
        self.name = job_name
        self.model = model
        self.steps = use_machines
        self.status = -1 #not assigned to machines yet
        self.current_machine = None 

    def get_model(self):
        return self.model
    
    def get_name(self):
        return self.name
    
    def info(self):
        return (self.name, self.status)
    
    def get_position(self):
        return self.status
    
    def update_machines(self, new_machines_assignment):
        self.steps = new_machines_assignment
    
    """
    def add_machine(self, step, new_machine):
        x = self.steps[step]
        x += [new_machine]
        self.steps[step] = x
        #print(self.name)
        print(len(self.steps[step]))
    """
    def get_next_machine(self):
        if self.status >= len(self.steps)-1:
            return None
        else:
            machines = self.steps[self.status+1]
        
        if len(machines) == 1: return machines[0]
        else:
            for m in machines:
                if m.free() and len(m.waitlist) == 0: return m
        machines.sort(key=lambda m: len(m.waitlist))
        if self.name == 'D25_15': print(machines[-1].name)
        return machines[-1]
    
    def move_to_next_machine(self):
        m = self.get_next_machine()
        if m:
            self.status += 1
            self.current_machine = m

## machine class
each machine requires input:
* machine name
* runtimes for each model (dictionary)
* setup times for each model (dictionary)

each machine has fields:
* machine name
* runtimes for each model
* setup times for each model
* the model the machine was/is currently working on, if any
* the job that is assigned to this machine or the job currently running, if any
* time to complete current job (or the job assigned to start in the future)
* the jobs waiting on this machine


In [3]:
class machine:
    def __init__(self, machine_name, runtimes, setup):
        self.name = machine_name
        self.runtimes = runtimes
        self.setup = setup
        self.status = 0 # 0 is not working, 0.5 is setup for new model, 1 is working
        self.previous_model = None
        self.current_job = None
        self.time_to_completion = 0
        self.waitlist = []
    
    def shutdown(self):
        self.status = 0
    
    def free(self):
        if not self.current_job or self.time_to_completion == 0: return True
        return False
    
    def get_current_job(self):
        return self.current_job
    
    def get_current_job_name(self):
        j = self.get_current_job()
        if j: return j.get_name()
        else: return ""
    
    def get_status(self):
        return self.status
    
    def get_time_to_completion(self):
        return self.time_to_completion
    
    def show_status(self):
        if self.status == 0.5: 
            return ("setup")
        elif self.status == 1:
            return ("working")
        else:
            return ("not working")
    
    def get_info(self):
        if self.status == 0:
            if self.current_job:
                print("machine {} is not working now, waiting to start job {}".format(self.name, self.current_job.get_name()))
                return (self.name, self.status, self.current_job.name, self.time_to_completion)
            else:
                print("machine {} is not working now".format(self.name))
                return (self.name, self.status,"", self.time_to_completion)
        else:
            print("machine {} is {} for model {}".format(self.name, self.show_status(), self.current_job.get_name()))
            return (self.name, self.status, self.current_job.name, self.time_to_completion)
    
    def get_waitlist(self):
        return self.waitlist
    
    
    def get_job_setup_time(self, model):
        return self.setup[model]
    
    
    def get_job_runtime(self, model):
        return self.runtimes[model]
    
    def if_in_use(self):
        if not self.current_job and self.time_to_completion <=0: 
            return False
        elif self.current_job and self.time_to_completion <=0:
            print("ERROR!!! machine {} is has job {} but has 0 or negative remaining runtime".format(self.name, self.current_job.name))
        return True
    
    def add_job(self, job, remaining_time_of_day):
        if self.current_job:
            self.waitlist.append(job)
        else:
            j = self.try_start_next_job(job, remaining_time_of_day)
            # if could not started the job, append it to waitlist
            if j and j not in self.waitlist: 
                self.waitlist.append(j)
            
    # ignore previous current_job and time to complete info and reset
    def start_job(self, job):
        
        if job in self.waitlist:
            self.waitlist.remove(job)
        self.current_job = job
        new_model = job.get_model()
        self.time_to_completion = self.get_job_runtime(new_model)
        self.previous_model = new_model
        self.status = 1
        #print("machine {} just started new job {}".format(self.name, job.get_name()))
    
    def complete_job(self):
        j = self.current_job
        #print("machine {} just completed job {}".format(self.name, j.name))
        self.current_job = None
        self.time_to_completion = 0
        self.status = 0
        return j
    
    def job_need_time(self, job):
        _, _, setup_time = self.need_setup(job)
        return self.get_job_runtime(job.get_model())+setup_time
    
    # helper function - find the fastest job (setup if needed+runtime) that is in this week's running models
    def find_fastest_job_in_waitlist(self, models_to_run):
        self.waitlist.sort(key = lambda j: self.job_need_time(j))
        for i in range(len(self.waitlist)):
            if self.waitlist[i].get_model() in models_to_run:
                return self.waitlist[i]
        return None
    
    # compare possible setup time and runtime, pick the shortest & first arrived one
    def get_next_job(self, models_to_run):
        if not self.waitlist: 
            return None
        elif len(self.waitlist) == 1:
            j = self.waitlist[0]
            if j.get_model() in models_to_run:
                return j
            else: return None
        else:
            # find the fastest job (setup if needed+runtime) that is in this week's running models
            j = self.find_fastest_job_in_waitlist(models_to_run)
            return j
    
    # input: job; output: (if_need_setup, new_model, setup_time)
    def need_setup(self, job):
        new_model = job.get_model()
        if self.previous_model and new_model != self.previous_model:
            return (True, new_model, self.get_job_setup_time(new_model))
        return (False, None, 0)
    
    def setup_new_model(self, job, new_model, setup_time):
        self.status = 0.5
        self.time_to_completion = setup_time
        self.previous_model = new_model
        self.current_job = job
    
    def try_start_next_job(self, j, remaining_time_of_day):
        if_need_setup, new_model, setup_time = self.need_setup(j)
        j_runtime = self.get_job_runtime(j.get_model())
        # need setup and have time to do it
        if if_need_setup and setup_time <= remaining_time_of_day:
            self.setup_new_model(j, new_model, setup_time)
            # have not started the job yet, return the job
            return j
        
        # does neet setup and have time to run it
        elif not if_need_setup and j_runtime <= remaining_time_of_day:
            self.start_job(j)

        # need setup and does not have time to do it, done for the day
        # or does not need setup but need more runtime than given
        else:
            self.status = 0
            self.current_job = None
            self.time_to_completion = 0
            # can't start the job, return the job
            return j
        
        return None
    
    # update machine status after time t
    def update(self, models_to_run, t, remaining_time_of_day):
        if self.current_job in self.waitlist:
            self.waitlist.remove(self.current_job)
        
        completed_job = None
        #if self.current_job:
        # the machine was not working, now could start work
        if self.status == 0:
            # start possible work assigned from yesterday
            if self.current_job and self.time_to_completion <= remaining_time_of_day:
                self.start_job(self.current_job)
            # or if nothing was assigned, try start a new job
            elif not self.current_job:
                j = self.get_next_job(models_to_run)
                if j:
                    self.try_start_next_job(j, remaining_time_of_day)

            # if have an assigned job but has nothing to do
            else:
                pass
        
        # check if completed current job
        elif self.time_to_completion - t <= 0 and self.status == 1:
            completed_job = self.complete_job()
            j = self.get_next_job(models_to_run)
            if j:
                self.try_start_next_job(j, remaining_time_of_day)


        # if just finished setup, check if having time for next operation
        elif self.time_to_completion - t <= 0 and self.status == 0.5:
            # if have time to run model:
            j_runtime = self.get_job_runtime(self.current_job.get_model())
            if j_runtime <= remaining_time_of_day:
                self.time_to_completion = 0
                self.start_job(self.current_job)
            else:
                #not have time to run model
                self.status = 0
                self.time_to_completion = self.get_job_runtime(self.current_job.get_model())

        else:
            self.time_to_completion -= t


        return completed_job
            

## factory class
requires input:
* machines
* how many hours is this factory working per day
* (optional) initial jobs   <-- some bugs here, please use release_jobs function instead

In [4]:
class factory:
    def __init__(self, machines, working_hours, jobs = set()):
        self.machines = machines
        self.jobs = jobs
        self.working_hours = working_hours
    
    def add_machine(self, machine):
        self.machines.append(machine)
        
    def set_working_hours(self, working_hours):
        self.working_hours = working_hours
    
    def get_status(self):
        df = pd.DataFrame(columns=["machines","status","assigned_job","time_to_completion","num_jobs_waiting"])
        df['machines'] = [m.name for m in self.machines]
        df['status'] = [m.show_status() for m in self.machines]
        df['assigned_job'] = [m.get_current_job_name() for m in self.machines]
        df['time_to_completion'] = [m.get_time_to_completion() for m in self.machines]
        df['num_jobs_waiting'] = [len(m.get_waitlist()) for m in self.machines]
         
        return df
    
    # only balance waitlist jobs at the end of day
    def balance_jobs_between_two_machines(self, machine1, machine2):
        if machine1.waitlist or machine2.waitlist:
            if len(machine1.waitlist) > len(machine2.waitlist):
                idx = int((len(machine1.waitlist)+len(machine2.waitlist))/2)
                w1 = machine1.waitlist
                w2 = machine2.waitlist
                if len(w1[idx:]) > 0: 
                    w2 = list(w2+w1[idx:])
                machine1.waitlist = list(set(w1[:idx]))
                machine2.waitlist = list(set(w2))
            elif len(machine1.waitlist) < len(machine2.waitlist):
                idx = int((len(machine1.waitlist)+len(machine2.waitlist))/2)
                w1 = machine1.waitlist
                w2 = machine2.waitlist
                if len(w2[idx:]) > 0: 
                    w1 = list(w1+w2[idx:])
                machine2.waitlist = list(set(w2[:idx]))
                machine1.waitlist = list(set(w1))
        
        
    def print_waiting_jobs(self):
        for m in self.machines:
            w = [j.get_name() for j in m.waitlist]
            if m.get_current_job_name(): w.append(m.get_current_job_name())
            print("machine {} jobs waitlist: {}".format(m.name,w))
    
    def release_jobs(self, jobs, remaining_time_of_day):
        finished_products = []
        for j in jobs:
            if j not in self.jobs: self.jobs.add(j)
            m = j.get_next_machine()
            
            if m: 
                j.move_to_next_machine()
                m.add_job(j, remaining_time_of_day)
            else:
                # completed all steps
                if j in self.jobs: self.jobs.remove(j)
                finished_products.append(j.get_name())
                #print("job {} has finished all steps.".format(j.get_name()))
        
        # return list of finished jobs
        return finished_products
    
    def get_all_jobs(self):
        return self.jobs
    
    def print_finished_products(self, finished_products):
        finished_products_dic = {}
        for p in finished_products:
            if p[:3] in finished_products_dic: finished_products_dic[p[:3]] += 1
            else: finished_products_dic[p[:3]] = 1
        #print(finished_products_dic)
        return finished_products_dic
    
    def workday(self, models_to_run, remaining_time_of_day = None, t = 0.5):
        if not remaining_time_of_day: remaining_time_of_day = self.working_hours
        setup_times = dict.fromkeys([m.name for m in self.machines], 0)
        runtimes = dict.fromkeys([m.name for m in self.machines], 0)
        finished_products= []
        while remaining_time_of_day > 0:
            completed_jobs = []
            for m in self.machines:
                # keep track of runtime and setup times
                if m.get_status() == 1: 
                    runtimes[m.name] += t
                elif m.get_status() == 0.5: 
                    setup_times[m.name] += t
                
                j = m.update(models_to_run, t, remaining_time_of_day)
                if j: 
                    completed_jobs.append(j)
                    #print("machine {} completed job {} at time {}".format(m.name, j.get_name(), self.working_hours-remaining_time_of_day))
                
                # debug
                #if m == c1 and j:
                #    print(remaining_time_of_day-t)
                #    print(j.name)
                #if m==c1:
                #    print(len(m.waitlist))
                
            finished_products += self.release_jobs(completed_jobs, remaining_time_of_day)
            
            remaining_time_of_day -= t
        
        # shutdown all machines at the end of day
        for m in self.machines:
            m.shutdown()
        
        #print today's finished products (jobs that completed all steps)
        #print("finished products : {}".format(finished_products))
        
        
        finished_products_dic = self.print_finished_products(finished_products)
        #print("finished products : {}".format(finished_products_dic))
        
        #print job waitlist for each machine
        #self.print_waiting_jobs()
        
        """
        nd1_waiting_jobs = {}
        for j in nd1.waitlist:
            if j.get_model() in nd1_waiting_jobs:
                nd1_waiting_jobs[j.get_model()]+=1
            else:
                nd1_waiting_jobs[j.get_model()]=1
                
        print("waiting : {}".format(nd1_waiting_jobs))
        """
        #print("machines {}".format([m.name for m in self.machines]))
        #print("total setup time for each machine: {}".format(setup_times.values()))
        #print("total runtime for each machine: {}".format(runtimes.values()))
            
        return (finished_products_dic, setup_times, runtimes)
    
    
    
    

## a helper function to create jobs

In [5]:
# create jobs with name format "model_x" where x = 1,2, ...
def create_job(name, machines_dict, model=None):
    if not model: model = name[:3]
    return job(name, model, machines_dict[model])

def create_job_list(model, number, machines_dict, wk = 0):
    return [create_job(model+"_"+str(i)+"_wk"+str(wk), machines_dict,model) for i in range(number)]

# takes demands (dictionary object) as input
def create_job_list_by_demands(demands, machines_dict,wk = 0):
    ls = []
    for model, number in demands.items():
        ls.append(create_job_list(model, number, machines_dict, wk))
    return ls

# example usage: 
# demands = {"B15":7, "D25":5, "C17":14}
# create_job_list_by_demands(demands, 1)
# print([([i.name for i in ls]) for ls in create_job_list_by_demands(demands, 1)])

### helper function to print useful informations

In [6]:
def merge_dict(dictionary, big_dictionary={}):
    for key,value in dictionary.items():
        if key in big_dictionary.keys(): big_dictionary[key]+=value
        else: big_dictionary[key]=value
    return big_dictionary

### helper function to change layouts

In [7]:
def update_machines_assignment(factory, machines_dict2):
    # update machines assignment to all existing jobs in factory
    for j in factory.get_all_jobs():
        new_machines = machines_dict2[j.get_model()]
        j.update_machines(new_machines)

# if purchased new machine, remember to add to list of machines in factory object
def add_machine_to_factory(factory, machine):
    factory.add_machine(machine)

# How to use this model:

## 1. define the settings for machines and models

In [8]:
# settings
# chucker 1, 2, 3
runtimes1 = {"D20":4.5,"D25":4.5,"B15":6,"E26":6,"C17":3}
setup1 = {"D20":1.5,"D25":1.5,"B15":1.5,"E26":1.5,"C17":1.5}
c1,c2,c3 = machine("Chucker1", runtimes1, setup1),machine("Chucker2", runtimes1, setup1),machine("Chucker3", runtimes1, setup1)

# need change following values
runtimes2 = {"D20":8,"D25":8,"B15":8,"E26":9,"C17":8}
setup2 = {"D20":1.5,"D25":1.5,"B15":1.5,"E26":1.5,"C17":1.5}
ca,cb,cc,cd,ce= machine("ChuckerA", runtimes2, setup2),machine("ChuckerB", runtimes2, setup2),machine("ChuckerC", runtimes2, setup2),machine("ChuckerD", runtimes2, setup2),machine("ChuckerE", runtimes2, setup2)


runtimes3 = {"D20":3,"D25":3,"B15":4,"E26":4.5,"C17":4}
setup3 = {"D20":2,"D25":2,"B15":2,"E26":2,"C17":2}
m1, m2 = machine("Mill1", runtimes3, setup3),machine("Mill2", runtimes3, setup3)

runtimes4 = {"D20":2,"D25":2,"B15":1.5,"E26":2,"C17":1.5,"F35":3}
setup4 = {"D20":3,"D25":3,"B15":3,"E26":3,"C17":3, "F35":3}
nd1 = machine("Naco Drill", runtimes4, setup4)
nd2 = machine("Naco Dril2", runtimes4, setup4)

# create list for all machies for  initial setting
machines_init = [c1,c2,c3,ca,cb,cc,cd,m1,m2,nd1]

# models of the jobs
models = ["D20","D25","B15","E26","C17","F35"]

# create the pairs we might want to share jobs
# **** not currently in use! ****
machine_pairs_to_balance_0 = [(ca,cb)]

In [9]:
# create all the possible layouts we could use in the future
models_to_run_before=["D20","D25","B15","E26","C17","F35"]
machines_dict_before = {"D25":[[c3], [cd], [m2], [nd1]], 
                "D20":[[c3], [cc], [m2], [nd1]],
                "B15":[[c3], [cd],[m2], [nd1]],
                "E26":[[c2], [cc], [m2], [nd1]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layout_before = (models_to_run_before, machines_dict_before,[])


models_to_run_init = ["B15","C17","F35"]
#balance_pair_init = [(cc, cd),(ca, cb),(c3,c2)]
balance_pair_init =[(cc,cd),(c2,c3)]
machines_dict_init = {"D25":[[c2], [cb,cc], [m1], [nd1]], 
                "D20":[[c3], [cd], [m2], [nd1]],
                "B15":[[c2,c3], [cc,cd],[m2],[nd1]],
                "E26":[[c3], [cb],[m1],[nd1]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layout0 = (models_to_run_init,machines_dict_init,balance_pair_init)

models_to_run_wk3 = ["B15","F35","C17"]
machines_dict2 = {"D25":[[c2], [cc], [m1], [nd1,nd2]], 
                "D20":[[c3], [cd], [m2], [nd1,nd2]],
                "B15":[[c2,c3], [cc,cd],[m2],[nd1, nd2]],
                "E26":[[c3], [cb],[m1],[nd1,nd2]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layout1 = (models_to_run_wk3, machines_dict2, balance_pair_init)

# test layouts for week 3 - 12
# A
# NOT running D25, E26
"""
balance_pair_A = [(ca, cb)]
models_to_run_A = ["B15","C17","D20", "F35"]
machines_dict_A = {"D25":[[c2], [cb,cc], [m1,m2], [nd1]], 
                "D20":[[c2], [cc], [m2], [nd1]],
                "B15":[[c3], [cd],[m2],[nd1]],
                "E26":[[c2], [cb,cc],[m1,m2],[nd1]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layoutA = (models_to_run_A, machines_dict_A, balance_pair_A)

"""
#balance_pair_A = [(ca, cb),(c1,c2)]
balance_pair_A = [(c1, c2),(cb,cc)]
models_to_run_A = ["B15", "C17", "F35","D25"]
machines_dict_A = {"D25":[[c1,c2], [cb,cc], [m1], [nd2]], 
                "D20":[[c2], [cc], [m2], [nd2]],
                "B15":[[c3], [cc, cd], [m2],[nd2]],
                "E26":[[c1,c2], [cb,cc],[m1,m2],[nd1,nd2]],
                "C17":[[c1], [ca],[m1],[nd1]],
                "F35":[[nd1]]}
layoutA = (models_to_run_A, machines_dict_A, balance_pair_A)

# B
# not running D25, D20, F35
#balance_pair_B = [(ca,cb),(c1,c2)]
balance_pair_B = []
models_to_run_B = ["B15","C17","E26", "F35"]
machines_dict_B = {"D25":[[c2], [cb,cc], [m1,m2], [nd1,nd2]], 
                "D20":[[c2], [cc], [m2], [nd2]],
                "B15":[[c3], [cd],[m2],[nd2]],
                "E26":[[c2], [cc],[m1,m2],[nd1,nd2]],
                "C17":[[c1], [ca],[m1],[nd1]],
                "F35":[[nd1]]}
layoutB = (models_to_run_B, machines_dict_B, balance_pair_B)
# C
# running D20
"""
balance_pair_C = [(cd, cc),(ca,cb)]
models_to_run_C = ["B15","C17","D25","F35"]
machines_dict_C = {"D25":[[c2], [cb,cc], [m1,m2], [nd1]], 
                "D20":[[c2], [cc], [m2], [nd1]],
                "B15":[[c3], [cd],[m2],[nd1]],
                "E26":[[c2], [cb,cc],[m1,m2],[nd1]],
                "C17":[[c1], [ca], [m1],[nd1]],
                "F35":[[nd1]]}
layoutC = (models_to_run_C, machines_dict_C, balance_pair_C)

"""
#balance_pair_C = [(cd, cc),(ca,cb),(c1,c2)]
balance_pair_C=[(c1,c2),(ca,cb),(cc, cd)]
models_to_run_C = ["B15","C17","D20","F35"]
machines_dict_C = {"D25":[[c1,c2], [cb,cc], [m1,m2], [nd1,nd2]], 
                "D20":[[c1,c2], [cb,cc], [m1], [nd2]],
                "B15":[[c3], [cc,cd],[m2],[nd2]],
                "E26":[[c1,c2], [cb,cc],[m1],[nd1,nd2]],
                "C17":[[c1], [ca], [m1],[nd1]],
                "F35":[[nd1]]}
layoutC = (models_to_run_C, machines_dict_C, balance_pair_C)


# test layouts for week 13-16
# new chucker ce
# A
models_to_run_A2 = ["B15","C17","D25"]
machines_dict_A2 = {"D25":[[c2], [cc], [m2], [nd2]], 
                "D20":[[c2], [cc], [m2], [nd2]],
                "B15":[[c3], [cd, ce],[m2],[nd2]],
                "E26":[[c2], [cc],[m2],[nd2]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layoutA2 = (models_to_run_A2, machines_dict_A2, balance_pair_A)

# C2
balance_pair_C2 = [(ca, cb), (cc,cd)]
models_to_run_C2 = ["B15","C17","E26"]
machines_dict_C2 = {"D25":[[c2], [cc], [m2], [nd2]], 
                "D20":[[c2], [cc,cd], [m2], [nd2]],
                "B15":[[c3], [ce],[m2],[nd2]],
                "E26":[[c2], [cc],[m2],[nd2]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layoutC2 = (models_to_run_C2, machines_dict_C2, balance_pair_C2)

# D - run all
balance_pair_D = [(ca, cb),(cc,cd), (m1,m2)]
models_to_run_D = ["B15", "C17","E26", "D25", "D20","F35"]
machines_dict_D = {"D25":[[c2], [cc,cb], [m2], [nd2]], 
                "D20":[[c2], [cc,cd], [m2], [nd2]],
                "B15":[[c3], [ce], [m2],[nd2]],
                "E26":[[c2], [cc],[m2],[nd2]],
                "C17":[[c1], [ca,cb],[m1],[nd1]],
                "F35":[[nd1]]}
layoutD = (models_to_run_D, machines_dict_D, balance_pair_D)

In [10]:
# define how many hours is the factory working right now
work_hours = 16

# initialize jobs with backlog, using the initial layout
backlogs_0 = []
B15_jobs_0 = [create_job("B15_backlog"+str(i), machines_dict_before) for i in range(7)]
D25_jobs_0 = [create_job("D25_backlog"+str(j), machines_dict_before) for j in range(5)]
C17_jobs_0 = [create_job("C17_backlog"+str(k), machines_dict_before) for k in range(14)]
D20_jobs_0 = [create_job("D20_backlog"+str(i), machines_dict_before) for i in range(4)]
E26_jobs_0 = [create_job("E26_backlog"+str(j), machines_dict_before) for j in range(4)]
F35_jobs_0 = [create_job("F35_backlog"+str(k), machines_dict_before) for k in range(3)]

backlogs_0 = [B15_jobs_0, D25_jobs_0, C17_jobs_0,D20_jobs_0,E26_jobs_0,F35_jobs_0]

## 2. initialize the factory with backlogs

In [11]:

my_factory = factory(machines_init, work_hours)

# release all backlog jobs:
for ls in backlogs_0:
    my_factory.release_jobs(ls, 16)


## 3. function: open my factory for a week
input:
* week (int)
* factory object
* how many hours is the factory working everyday
* demands for each model this week
* this week's machines assignment for each model
* how many days are we working for a week (default = 5)

output:
* finished products
* total time spent on setup by machine
* total runtime by machine

In [12]:
def open_factory_week(week, factory, work_hours, demands, layout, total_finished_products = {}, 
                      total_setup_times = {}, total_runtimes = {},new_machine_assignment=False, 
                      days_working=5):
    
    models_to_run, machine_assignment, balance_pairs = layout
    
    # if has a new machine assignement, do it
    if new_machine_assignment: 
        update_machines_assignment(factory, machine_assignment)
        
    # release new demands
    for ls in create_job_list_by_demands(demands, machine_assignment,week):
        my_factory.release_jobs(ls,work_hours)
    
    # start work!
    for i in range(days_working):
        #print("day " + str(i+1))
        finished_products, setup_times, runtimes = factory.workday(models_to_run)
        total_finished_products = merge_dict(finished_products, total_finished_products)
        total_setup_times = merge_dict(setup_times, total_setup_times)
        total_runtimes = merge_dict(runtimes, total_runtimes)
        
        # test
        #if balance_pairs:
        #    for pair in balance_pairs:
        #        factory.balance_jobs_between_two_machines(pair[0], pair[1])
        
    #print("week " + str(week))
    #print("finished products : {}".format(total_finished_products))
    #print("total setup time : {}".format(total_setup_times))
    #print("total runtime : {}".format(total_runtimes))
    
    df_product = pd.DataFrame.from_dict(total_finished_products,orient='index').T
    df_setup_times = pd.DataFrame.from_dict(total_setup_times,orient='index').T
    df_runtime = pd.DataFrame.from_dict(total_runtimes,orient='index').T
    df_product["week"] = week
    df_runtime["week"] = week
    df_setup_times["week"] = week
    
    return (df_product, df_setup_times, df_runtime)

# run the model

* provide week number
* set the weekly new demand
* set working hours everyday
* provide any layout changes if needed

use function:
open_factory_week(week, factory, work_hours, demands, total_finished_products = {}, 
                      total_setup_times = {}, total_runtimes = {},new_machine_assignment=None, 
                      days_working=5)

In [13]:
# can't handle 2.5 demand, so set 1 week 2 next week 3 and iterate
df0 = pd.DataFrame(columns = ["B15","C17","D20","D25","E26","F35"])
df1 = pd.DataFrame(columns = ['Chucker1', 'Chucker2', 'Chucker3', 'ChuckerA', 'ChuckerB', 'ChuckerC', 'ChuckerD', 'Mill1', 'Mill2', 'Naco Drill', 'Naco Dril2'])
df2 = pd.DataFrame(columns = ['Chucker1', 'Chucker2', 'Chucker3', 'ChuckerA', 'ChuckerB', 'ChuckerC', 'ChuckerD', 'Mill1', 'Mill2', 'Naco Drill', 'Naco Dril2'])

def append_all_info(df_product, df_setup_times, df_runtime, df0=df0, df1=df1, df2=df2):
    df0 = df0.append(df_product)
    df1 = df1.append(df_setup_times)
    df2 = df2.append(df_runtime)
    return (df0, df1, df2)

demands_0 = {"B15":7, "C17":14, "D20":4, "D25":5, "E26":4, "F35": 2}
demands_1 = {"B15":7, "C17":14, "D20":4, "D25":5, "E26":4, "F35": 3}
# week 7-8
demands_2 = {"B15":7, "C17":14, "D20":4, "D25":5, "E26":4, "F35": 3}
demands_3 = {"B15":8, "C17":14, "D20":4, "D25":5, "E26":4, "F35": 2}
# start week 9-12
demands_4 = {"B15":7, "C17":14, "D20":5, "D25":5, "E26":3, "F35": 3}
demands_5 = {"B15":8, "C17":14, "D20":4, "D25":5, "E26":4, "F35": 2}

# start_week 13
demands_6 = {"B15":8, "C17":14, "D20":5, "D25":5, "E26":4, "F35": 3}

In [14]:
# run the simulation for previous 5 weeks
# initialize jobs with backlog, using the initial layout

"""
work_hours = 16
my_factory = factory(machines_init, work_hours)

# release all backlog jobs:
for ls in backlogs:
    my_factory.release_jobs(ls, work_hours)

"""
#work_hours_0 = 16

#my_factory.print_waiting_jobs()

"""
df_product, df_setup_times, df_runtime = open_factory_week(-4, my_factory, work_hours_0, demands_1, layout_before, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)


df_product, df_setup_times, df_runtime = open_factory_week(-3, my_factory, work_hours_0, demands_1, layout_before, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

df_product, df_setup_times, df_runtime = open_factory_week(-2, my_factory, work_hours_0, demands_1, layout_before, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

#my_factory.print_waiting_jobs()

df_product, df_setup_times, df_runtime = open_factory_week(-1, my_factory, work_hours_0, demands_0, layout_before, {},{},{}, new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
"""
#my_factory.print_waiting_jobs()
work_hours_0=16
df_product, df_setup_times, df_runtime = open_factory_week(0, my_factory, 16, demands_1, layout_before, {},{},{}, new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

#my_factory.print_waiting_jobs()

#print(df_runtime)

/Library/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [15]:
# our working start

backlogs = []
B15_jobs = [create_job("B15_backlog_2_"+str(i), machines_dict_before) for i in range(58)]
D25_jobs = [create_job("D25_backlog_2_"+str(j), machines_dict_before) for j in range(40)]
C17_jobs = [create_job("C17_backlog_2_"+str(k), machines_dict_before) for k in range(34)]
D20_jobs = [create_job("D20_backlog_2_"+str(i), machines_dict_before) for i in range(15)]
E26_jobs = [create_job("E26_backlog_2_"+str(j), machines_dict_before) for j in range(14)]
F35_jobs = [create_job("F35_backlog_2_"+str(k), machines_dict_before) for k in range(3)]
backlogs = [B15_jobs, D25_jobs, C17_jobs,D20_jobs,E26_jobs,F35_jobs]
"""
backlogs2 = []
B15_jobs2 = [create_job("B15_backlog_22_"+str(i), machines_dict_before) for i in range(58)]
D25_jobs2 = [create_job("D25_backlog_22_"+str(j), machines_dict_before) for j in range(40)]
C17_jobs2 = [create_job("C17_backlog_22_"+str(k), machines_dict_before) for k in range(34)]
D20_jobs2 = [create_job("D20_backlog_22_"+str(i), machines_dict_before) for i in range(15)]
E26_jobs2 = [create_job("E26_backlog_22_"+str(j), machines_dict_before) for j in range(14)]
F35_jobs2 = [create_job("F35_backlog_22_"+str(k), machines_dict_before) for k in range(3)]
backlogs2 = [B15_jobs2, D25_jobs2, C17_jobs2,D20_jobs2,E26_jobs2,F35_jobs2]
# release all backlog jobs:
"""

work_hours = 20
my_factory.working_hours = work_hours

for ls in backlogs:
    #print(ls)
    my_factory.release_jobs(ls, work_hours)



# first 2 weeks
#for i in range(1,3):
    #_,_,_ = open_factory_week(1, my_factory, 20, demands)  # for cumalitave output
    #_,_,_ = open_factory_week(i, my_factory, 20, demands_0, layout0, {},{},{},new_machine_assignment=False)

# first 2 weeks
df_product, df_setup_times, df_runtime = open_factory_week(1, my_factory, work_hours, demands_0, layout0, {},{},{},new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

df_product, df_setup_times, df_runtime = open_factory_week(2, my_factory, work_hours, demands_1, layout0, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

"""
# without 2nd nacho drill

# week 3 - 12
# using layout A for week 3-12
# week 3, undate the machines assignment, remember to add the new machine nd2
df_product, df_setup_times, df_runtime = open_factory_week(3, my_factory, work_hours, demands_0, layoutA, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 4-7, continue using layout A
for i in range(4, 7, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_1, layoutA, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_0, layoutA, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 8, undate the machines assignment
df_product, df_setup_times, df_runtime = open_factory_week(8, my_factory, work_hours, demands_1, layoutC, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 8-12, continue using layout C
for i in range(9, 12, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_0, layoutC, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    #my_factory.print_waiting_jobs()
    
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_1, layoutC, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    #my_factory.print_waiting_jobs()

# week 13-16
# week 13
add_machine_to_factory(my_factory, ce)
df_product, df_setup_times, df_runtime = open_factory_week(13, my_factory, work_hours, demands_0, layoutD, {},{},{}, new_machine_assignment=True)
#print(nd1.waitlist)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 14-16
for i in range(14, 17, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_1, layoutD, {},{},{},new_machine_assignment=False)
    #print(nd1.waitlist)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_0, layoutD, {},{},{},new_machine_assignment=False)
    #print(nd1.waitlist)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)


# week 17-30
    
    
"""
"""
# with 2nd nacho drill
# week 3 - 12
# using layout A for week 3-12
# week 3, undate the machines assignment, remember to add the new machine nd2
add_machine_to_factory(my_factory, nd2)
df_product, df_setup_times, df_runtime = open_factory_week(3, my_factory, work_hours, demands_0, layoutA, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 4-7, continue using layout A
for i in range(4, 7, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_1, layoutA, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_0, layoutA, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 8, undate the machines assignment
df_product, df_setup_times, df_runtime = open_factory_week(8, my_factory, work_hours, demands_1, layoutC, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 8-12, continue using layout C
for i in range(9, 12, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_0, layoutC, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    my_factory.print_waiting_jobs()
    
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_1, layoutC, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    my_factory.print_waiting_jobs()

# week 13-16
# week 13
add_machine_to_factory(my_factory, ce)
df_product, df_setup_times, df_runtime = open_factory_week(13, my_factory, work_hours, demands_0, layoutD, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 14-16
for i in range(14, 17, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_1, layoutD, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_0, layoutD, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
"""

# with 2nd nacho drill
# week 3 - 12

# using layout A for week 3-5
# week 3, undate the machines assignment, remember to add the new machine nd2
add_machine_to_factory(my_factory, nd2)
df_product, df_setup_times, df_runtime = open_factory_week(3, my_factory, work_hours, demands_0, layout1, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

df_product, df_setup_times, df_runtime = open_factory_week(4, my_factory, work_hours, demands_0, layout1, {},{},{}, new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 5, change to layout A
df_product, df_setup_times, df_runtime = open_factory_week(5, my_factory, work_hours, demands_0, layoutA, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 6, layout A
df_product, df_setup_times, df_runtime = open_factory_week(6, my_factory, work_hours, demands_1, layoutA, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 7, change to layout C
df_product, df_setup_times, df_runtime = open_factory_week(7, my_factory, work_hours, demands_1, layoutC, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 8, layout C
df_product, df_setup_times, df_runtime = open_factory_week(8, my_factory, work_hours, demands_0, layoutC, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

"""
# week 7, layout C
df_product, df_setup_times, df_runtime = open_factory_week(7, my_factory, work_hours, demands_2, layoutC, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
"""

# week 9, change to layout B
df_product, df_setup_times, df_runtime = open_factory_week(9, my_factory, work_hours, demands_3, layoutB, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
"""
# week 9, layout B
df_product, df_setup_times, df_runtime = open_factory_week(9, my_factory, work_hours, demands_4, layoutB, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
"""
#week 10, layout B
df_product, df_setup_times, df_runtime = open_factory_week(10, my_factory, work_hours, demands_5, layoutB, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 11 change to layout D
df_product, df_setup_times, df_runtime = open_factory_week(11, my_factory, work_hours, demands_4, layoutD, {},{},{},new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

#week 12 layout D
df_product, df_setup_times, df_runtime = open_factory_week(11, my_factory, work_hours, demands_5, layoutD, {},{},{},new_machine_assignment=False)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)


# week 13-16
# week 13
add_machine_to_factory(my_factory, ce)
df_product, df_setup_times, df_runtime = open_factory_week(13, my_factory, work_hours, demands_6, layoutD, {},{},{}, new_machine_assignment=True)
df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)

# week 14-16
for i in range(14, 17, 2):
    df_product, df_setup_times, df_runtime = open_factory_week(i, my_factory, work_hours, demands_6, layoutD, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)
    
    df_product, df_setup_times, df_runtime = open_factory_week(i+1, my_factory, work_hours, demands_6, layoutD, {},{},{},new_machine_assignment=False)
    df0, df1, df2 = append_all_info(df_product, df_setup_times, df_runtime, df0, df1, df2)





In [16]:
print(df0.fillna(0).set_index("week"))

      B15  C17  D20  D25  E26  F35
week                              
0.0     4    8    0    0    4    6
1.0     9   20    0    0    0    5
2.0    10   20    0    0    0    3
3.0    11   18    0    0    0    2
4.0    10   14    0    0    0    2
5.0    10   12    1    8    0    2
6.0    10   10    0   10    0    3
7.0     6   12    7    0    0    3
8.0     1   10   10    0    0    2
9.0     8   16    1    0    5    2
10.0   10   20    0    0    0    2
11.0    3   20    4    4    8    3
11.0    0    3   13    6   14    2
13.0   14   19    5    5    5    3
14.0    8   20    4    7    3    3
15.0    6   19   10    3    5    3
16.0   20   14    0    0    0    3
17.0    8   14   10    0    3    3


In [17]:
df0.fillna(0).set_index("week").to_csv("products.csv")

In [18]:
# total setup time
print(df1.fillna(0).set_index("week"))

      Chucker1  Chucker2  Chucker3  ChuckerA  ChuckerB  ChuckerC  ChuckerD  \
week                                                                         
0.0        0.0       0.0       1.5       0.0       0.0       0.0       0.0   
1.0        0.0       0.0       1.5       0.0       0.0       0.0       0.0   
2.0        0.0       1.5       0.0       0.0       0.0       0.0       0.0   
3.0        0.0       0.0       0.0       0.0       0.0       1.5       0.0   
4.0        0.0       0.0       0.0       0.0       0.0       0.0       0.0   
5.0        0.0       3.0       1.5       0.0       1.5       1.5       0.0   
6.0        0.0       3.0       3.0       0.0       0.0       0.0       0.0   
7.0        0.0       1.5       3.0       0.0       3.0       0.0       0.0   
8.0        0.0       0.0       3.0       0.0       0.0       0.0       0.0   
9.0        0.0       1.5       1.5       0.0       1.5       1.5       0.0   
10.0       0.0       3.0       0.0       0.0       0.0       0.0

In [19]:
# total runtims
print(df2.fillna(0).set_index("week"))

      Chucker1  Chucker2  Chucker3  ChuckerA  ChuckerB  ChuckerC  ChuckerD  \
week                                                                         
0.0       75.0      48.0      60.0      40.0      70.0      45.0      70.0   
1.0       90.0       6.0      90.0      80.0      80.0       0.0      80.0   
2.0       90.0       4.5      90.0      80.0      80.0       0.0      80.0   
3.0       57.0       4.5      90.0      56.0      80.0       8.0      80.0   
4.0       42.0       4.5      90.0      39.5      80.0       8.0      80.0   
5.0       42.0      85.5      87.0      80.0      80.0       8.0      80.0   
6.0       42.0      45.0      87.0      80.0      80.0       8.0      80.0   
7.0       42.0      45.0      87.0      80.0      80.0       0.0      40.0   
8.0       42.0      18.0      87.0      80.0      80.0       0.0       8.0   
9.0       42.0      88.5      90.0      80.0      80.0      90.0      72.0   
10.0      42.0      88.5      90.0      80.0      80.0      90.0

In [20]:
df1.to_csv("setup.csv")

In [21]:
df2.to_csv("runtime.csv")